**Bayesian Linear Regression in PYMC3 Tutorial**

This is a very simple tutorial on using PYMC3 for Bayesian linear regression, with an estimation of the posterior probability distributions.  It was adopted from the PYMC3 getting started documentation [https://docs.pymc.io/notebooks/getting_started.html](https://docs.pymc.io/notebooks/getting_started.html).

In [ ]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

Let's create some data for our regression.  Our true values are:
* $\alpha = 1$
* $\sigma = 1$
* $\beta = [1, 2.5]$

Our outcome variable is:
$$ Y = \alpha + \beta_1 X_1 + \beta_2 X_2 + N(0,\sigma).$$

In [ ]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1
beta = [1, 2.5]

# Size of dataset
size = 1000

# Predictor variable
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(size)*sigma

Here is a quick plot of our data.

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True, figsize=(10,4))
axes[0].scatter(X1, Y)
axes[1].scatter(X2, Y)
axes[0].set_ylabel('Y'); axes[0].set_xlabel('X1'); axes[1].set_xlabel('X2');

Now we build the model.

The prior distributions are:
* $\alpha \sim \mathcal{N}(\mu=0,\sigma=10)$
* $\beta[i] \sim \mathcal{N}(\mu=0,\sigma=10)$, where $i=1,2$
* $\sigma \sim \textrm{half-normal}(\sigma=1)$

We define the unobserved variable rate, which is a function of the year:
\begin{equation}
  \mu = \alpha + \beta[0]*X1 + \beta[1]*X2
\end{equation}

Our likelihood function in Bayes theorem is a Poisson distribution on the number of disasters, each year.  This can be written as:
\begin{equation}
  \text{Y}\sim \mathcal{N}(\mu=\mu,\sigma=\sigma).
\end{equation}

In [ ]:
basic_model = pm.Model()

with basic_model:

    # Priors for unknown model parameters
    alpha = pm.Normal('alpha', mu=0, sigma=100)
    beta = pm.Normal('beta', mu=0, sigma=100, shape=2)
    sigma = pm.HalfNormal('sigma', sigma=100)

    # Expected value of outcome
    mu = alpha + beta[0]*X1 + beta[1]*X2

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=Y)

Now we take 5000 random MCMC samples.  The defualt PYMC3 sampler is the Hamiltonian MC No U-Turn Sampler (NUTS), which is almost always a good choice.

In [ ]:
with basic_model:
    # draw 5000 posterior samples
    trace = pm.sample(5000)

The traceplot is the standard good way to view the posterior probability distributions along with theMCMC samples.

In [ ]:
pm.traceplot(trace);

There is a built-in summary function as well.

In [ ]:
pm.summary(trace).round(2)

Here are some 2-d plots of joint probability distributions.

In [ ]:
import seaborn as sns
plt.figure(figsize=(9,7))
sns.jointplot(trace['beta'][:,0], trace['beta'][:,1], kind="hex", color="#4CB391")
plt.xlabel("beta[0]")
plt.ylabel("beta[1]");
plt.show()

plt.figure(figsize=(9,7))
sns.jointplot(trace['alpha'], trace['sigma'], kind="hex", color="#4CB391")
plt.xlabel("alpha")
plt.ylabel("sigma");
plt.show()